In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import seaborn as sns
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from tensorflow import keras
import pymongo

In [2]:
def get_training():
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    db = client['refined-database']
    a= list(db["match"].find({}))
    a = pd.DataFrame(a)
    a = a.drop(columns = a.columns[0])
    return a

In [3]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")

def int_to_dummies(i):
    # Create a label for the number given
    #Input: int The value of the label
    result = [0]*3
    result[int(i)] = 1
    return np.array(result)

def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=[
                      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall'),
                      tf.keras.metrics.AUC(name='auc')])

    return model



def prediction_logits (y,batch_size,nb_label):
    # Normalize the batch of label to obtain a percentage
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    #Out : Numpy array [batch_size][10] the input normalized
    
    y_sum = np.sum(y, axis=1)
    for i in range (0,batch_size):
        for j in range (0,nb_label):
            if (y_sum[i] != 0):
                y[i][j] = y[i][j]/y_sum[i]
    
    return y


In [4]:
df = get_training()
df = df[df.ID_FTR != -1]

target = df["ID_FTR"].apply(int_to_dummies)
features = df[df.columns[6:]]

In [5]:
x_test = features.iloc[3550:].values
y_test = target.iloc[3550:].values

x_train = features.iloc[:3550].values
y_train = target.iloc[:3550].values

In [6]:
x_train = x_train.astype(float)

In [7]:
y_train = np.asarray(list(y_train))
y_test = np.asarray(list(y_test))

In [8]:
y_train.shape

(3550, 3)

In [9]:
model = create_model()

checkpoint_path = "models/model1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.fit(x_train,y_train, epochs=20,validation_data=(x_test,y_test),
          callbacks=[cp_callback])

Train on 3550 samples, validate on 427 samples
Epoch 1/20
3424/3550 [===========================>..] - ETA: 0s - loss: 1.0848 - accuracy: 0.6626 - precision: 0.4478 - recall: 0.0526 - auc: 0.5824
Epoch 00001: saving model to models/model1.ckpt
3550/3550 [==============================] - 5s 2ms/sample - loss: 1.0847 - accuracy: 0.6623 - precision: 0.4450 - recall: 0.0535 - auc: 0.5828 - val_loss: 1.1147 - val_accuracy: 0.6604 - val_precision: 0.3750 - val_recall: 0.0281 - val_auc: 0.5260
Epoch 2/20
3520/3550 [============================>.] - ETA: 0s - loss: 1.0702 - accuracy: 0.6650 - precision: 0.4828 - recall: 0.0716 - auc: 0.6036
Epoch 00002: saving model to models/model1.ckpt
3550/3550 [==============================] - 1s 164us/sample - loss: 1.0707 - accuracy: 0.6648 - precision: 0.4810 - recall: 0.0713 - auc: 0.6030 - val_loss: 1.1120 - val_accuracy: 0.6635 - val_precision: 0.4231 - val_recall: 0.0258 - val_auc: 0.5214
Epoch 3/20
3456/3550 [============================>.] - ETA

3200/3550 [==========================>...] - ETA: 0s - loss: 1.0417 - accuracy: 0.6774 - precision: 0.5866 - recall: 0.1091 - auc: 0.6451
Epoch 00019: saving model to models/model1.ckpt
3550/3550 [==============================] - 1s 188us/sample - loss: 1.0458 - accuracy: 0.6761 - precision: 0.5731 - recall: 0.1104 - auc: 0.6392 - val_loss: 1.1177 - val_accuracy: 0.6550 - val_precision: 0.4138 - val_recall: 0.0843 - val_auc: 0.5317
Epoch 20/20
3296/3550 [==========================>...] - ETA: 0s - loss: 1.0423 - accuracy: 0.6779 - precision: 0.5927 - recall: 0.1077 - auc: 0.6435
Epoch 00020: saving model to models/model1.ckpt
3550/3550 [==============================] - 1s 169us/sample - loss: 1.0423 - accuracy: 0.6773 - precision: 0.5863 - recall: 0.1082 - auc: 0.6434 - val_loss: 1.1214 - val_accuracy: 0.6526 - val_precision: 0.4182 - val_recall: 0.1077 - val_auc: 0.5309


In [22]:
index = np.random.randint(0,107)
print(y_test[index])
print_logits(model.predict(x = x_test[index].reshape(1,14), batch_size = 1),1,3)

[0 0 1]
------------ 0 ------------
prédiction :  0 pourcentage de certitude : 32.3  %
prédiction :  1 pourcentage de certitude : 28.3  %
prédiction :  2 pourcentage de certitude : 39.5  %


In [24]:
y_pred = model.predict(x = x_train)
y_pred = np.argmax(y_pred, axis=1)
conf_mat = tf.math.confusion_matrix(y_train, y_pred)